In [2]:
import org.locationtech.jts.geom._
import org.apache.spark.sql.types._
import org.locationtech.geomesa.spark.jts._

// import spark.implicits._
// below hack
val spark2: SparkSession = spark
import spark2.implicits._
spark.withJTS

import org.apache.spark.graphx._
// To make some of the examples work we will also need RDD
import org.apache.spark.rdd.RDD

In [14]:
val edges = spark2.read.load("ffe_edges.parquet")
edges.cache()
edges.count()

edges = [id: int, near_id: int]


37052

In [77]:
// vertices.rdd.first()
val edge_rdd: RDD[Edge[Any]] = 
    edges.rdd.map(x => Edge(x.getInt(0), x.getInt(1), null))
edge_rdd.first()                       

edge_rdd = MapPartitionsRDD[422] at map at <console>:49


Edge(1732,1701,null)

In [15]:
val vertices = spark2.read.load("ffe_vertices.parquet")
vertices.cache()
vertices.count()

vertices = [_c0: int, TARGET_FID: int ... 15 more fields]


4000

In [78]:
// vertices.rdd.first()
val vert_rdd: RDD[(VertexId, (Int, String, Any, Any))] = 
    vertices.rdd.map(x => (x.getInt(0), (x.getInt(1), x.getString(2), x.get(9), null)))
vert_rdd.first()

vert_rdd = MapPartitionsRDD[423] at map at <console>:49


(0,(1,Houghton Bay,POLYGON ((1749551.504199982 5422104.252850056, 1749551.544950008 5422110.366449833, 1749556.703549862 5422110.332049847, 1749556.662749767 5422104.218400002, 1749551.504199982 5422104.252850056)),null))

In [79]:
// Build the initial Graph
val graph = Graph(vert_rdd, edge_rdd)

graph = org.apache.spark.graphx.impl.GraphImpl@189d4250


org.apache.spark.graphx.impl.GraphImpl@189d4250

In [90]:
val facts: RDD[String] =
  graph.triplets.map(triplet =>
    triplet.srcAttr._1 + " is the " + triplet.attr + " of " + triplet.dstAttr._1)
facts.take(50)

facts = MapPartitionsRDD[505] at map at <console>:48


Array(1 is the null of 845, 1 is the null of 850, 1 is the null of 851, 1 is the null of 852, 1 is the null of 853, 1 is the null of 854, 1 is the null of 855, 1 is the null of 856, 1 is the null of 857, 1 is the null of 1268, 1 is the null of 1269, 1 is the null of 1270, 1 is the null of 1279, 4 is the null of 744, 4 is the null of 745, 4 is the null of 746, 4 is the null of 747, 4 is the null of 755, 4 is the null of 756, 4 is the null of 757, 4 is the null of 758, 4 is the null of 759, 4 is the null of 763, 4 is the null of 764, 4 is the null of 765, 4 is the null of 767, 4 is the null of 769, 4 is the null of 771, 4 is the null of 776, 4 is the null of 777, 8 is the null of...

In [81]:
graph.numEdges

37052

In [82]:
graph.inDegrees.first()

(3558,19)

In [83]:
graph.triangleCount().vertices.take(10)

Array((3558,95), (1084,13), (3586,61), (3702,5), (3007,28), (667,3), (1053,3), (1894,21), (2493,22), (1325,30))

In [88]:
val sourceId: VertexId = 4 // The ultimate source
// Initialize the graph such that all vertices except the root have distance infinity.
val initialGraph = graph.mapVertices((id, _) =>
    if (id == sourceId) 0.0 else Double.PositiveInfinity)

sourceId = 4
initialGraph = org.apache.spark.graphx.impl.GraphImpl@47fbac06


org.apache.spark.graphx.impl.GraphImpl@47fbac06

In [93]:
initialGraph.numVertices

4000